In [1]:
#会员数据营销案例

In [22]:
import time
import numpy as np
import pandas as pd

In [3]:
dtypes = {'ORDERDATE':object, 'ORDERID':object, 'AMOUNTINFO': np.float32}#设置每列数据类型
raw_data = pd.read_csv('sales.csv',dtype=dtypes, index_col='USERID')
#数据概览
print('Data OVerview:')
print(raw_data.head(4))
print('-' * 30)
print('Data DESC:')
print(raw_data.describe())
print('-' * 60)

Data OVerview:
         ORDERDATE     ORDERID  AMOUNTINFO
USERID                                    
142074  2016-01-01  4196439032      9399.0
56927   2016-01-01  4198324983      8799.0
87058   2016-01-01  4191287379      6899.0
136104  2016-01-01  4198508313      5999.0
------------------------------
Data DESC:
         AMOUNTINFO
count  86127.000000
mean     744.762939
std     1425.194336
min        0.500000
25%       13.000000
50%       59.000000
75%      629.000000
max    30999.000000
------------------------------------------------------------


In [4]:
#缺失值审查
na_cols = raw_data.isnull().any(axis=0) #查看每一列的是否具有缺失值
print('NA Cols:')
print(na_cols)
print('-' * 30)
na_lines = raw_data.isnull().any(axis=1)
print('NA Recors:')
print('Total number of NA lines is: {0}'.format(na_lines.sum())) #查看具有缺失值得行总记录数
print(raw_data[na_lines]) #只查看具有缺失值的行信息
print('-' * 60)

NA Cols:
ORDERDATE      True
ORDERID       False
AMOUNTINFO     True
dtype: bool
------------------------------
NA Recors:
Total number of NA lines is: 10
         ORDERDATE     ORDERID  AMOUNTINFO
USERID                                    
75849   2016-01-01  4197103430         NaN
103714         NaN  4136159682       189.0
155209  2016-01-01  4177940815         NaN
139877         NaN  4111956196         6.3
54599   2016-01-01  4119525205         NaN
65456   2016-01-02  4195643356         NaN
122134  2016-09-21  3826649773         NaN
116995  2016-10-24  3981569421         NaN
98888   2016-12-06  3814398698         NaN
145951  2016-12-29  4139830098         NaN
------------------------------------------------------------


In [5]:
#异常值处理
sales_data = raw_data.dropna() #丢弃带有缺失值得行记录
sales_data = sales_data[sales_data['AMOUNTINFO'] > 1] #丢弃订单金额 <= 1的记录

In [6]:
#日期格式转换
sales_data['ORDERDATE'] = pd.to_datetime(sales_data['ORDERDATE'], format='%Y-%m-%d')
print('Raw Dtypes:')
print(sales_data.dtypes)
print('-' * 60)

Raw Dtypes:
ORDERDATE     datetime64[ns]
ORDERID               object
AMOUNTINFO           float32
dtype: object
------------------------------------------------------------


In [7]:
#数据转换
recency_value = sales_data['ORDERDATE'].groupby(sales_data.index).max() #计算原始最近一次订单时间
frequency_value = sales_data['ORDERDATE'].groupby(sales_data.index).count() #计算原始订单频率
monetary_value = sales_data['AMOUNTINFO'].groupby(sales_data.index).sum() #计算原始订单总金额

In [8]:
#计算RFM得分
deadline_date = pd.datetime(2017,1,1)
r_interval = (deadline_date - recency_value).dt.days #计算R间隔
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1]) #计算R得分
f_score = pd.cut(frequency_value, 5, labels=[1, 2, 3, 4, 5]) #计算F得分
m_score = pd.cut(monetary_value, 5, labels=[1,2,3,4,5]) #计算M得分

In [9]:
#R、F、M 数据合并
rfm_list = [r_score, f_score, m_score] #将r、f、m三个维度组成列表
rfm_cols = ['r_score','f_score','m_score'] #设置r、f、m 三个维度
rfm_pd = pd.DataFrame(np.array(rfm_list).transpose(),dtype=np.int32, columns=rfm_cols,index=frequency_value.index)
print('RFM Score Overview')
print(rfm_pd.head(4))
print('-' * 60)

RFM Score Overview
        r_score  f_score  m_score
USERID                           
51220         4        1        1
51221         2        1        1
51224         3        1        1
51225         4        1        1
------------------------------------------------------------


In [20]:
# 计算RFM总得分
# 方法一：加权得分
rfm_pd['rfm_wscore'] = rfm_pd['r_score'] * 0.6 + rfm_pd['f_score'] * 0.3 + rfm_pd['m_score'] * 0.1
# 方法二：RFM组合
rfm_pd_tmp = rfm_pd.copy()
rfm_pd_tmp['r_score'] = rfm_pd_tmp['r_score'].astype('str')
rfm_pd_tmp['f_score'] = rfm_pd_tmp['f_score'].astype('str')
rfm_pd_tmp['m_score'] = rfm_pd_tmp['m_score'].astype('str')
rfm_pd['rfm_comb'] = rfm_pd_tmp['r_score'].str.cat(rfm_pd_tmp['f_score']).str.cat(rfm_pd_tmp['m_score'])

In [21]:
# 打印输出和保存结果
# 打印结果
print ('Final RFM Scores Overview:')
print (rfm_pd.head(4))  # 打印数据前4项结果
print ('-' * 30)
print ('Final RFM Scores DESC:')
print (rfm_pd.describe())

Final RFM Scores Overview:
        r_score  f_score  m_score  rfm_wscore rfm_comb
USERID                                                
51220         4        1        1         2.8      411
51221         2        1        1         1.6      211
51224         3        1        1         2.2      311
51225         4        1        1         2.8      411
------------------------------
Final RFM Scores DESC:
            r_score       f_score       m_score    rfm_wscore
count  59676.000000  59676.000000  59676.000000  59676.000000
mean       3.299970      1.013439      1.000134      2.384027
std        1.402166      0.116017      0.018307      0.845380
min        1.000000      1.000000      1.000000      1.000000
25%        2.000000      1.000000      1.000000      1.600000
50%        3.000000      1.000000      1.000000      2.200000
75%        5.000000      1.000000      1.000000      3.400000
max        5.000000      5.000000      5.000000      5.000000


In [12]:
# 保存RFM得分到本地文件
rfm_pd.to_csv('sales_rfm_score.csv')  # 保存数据为csv